# Chain

## Review

We built a simple graph with nodes, normal edges, and conditional edges.

### Goals

Now, let's build up to a simple chain that combines 4 concepts:

* Using chat messages as our graph state
* Using chat models in graph nodes
* Binding tools to our chat model
* Executing tool calls in graph nodes

## Messages

Chat models can use _messages_, which capture different roles within a conversation.

LangChain supports various message types, including _HumanMessage_, _AIMessage_, _SystemMessage_, and _ToolMessage_.

These represent a message from the user, from chat model, for the chat model to instruct behavior, and from a tool call.

Let's create a list of messages.

Each message can be supplied with a few things:

* _content_ - content of the message
* _name_ - optionally, a message author
* _response_metadata_ - optionally, a dict of metadata (e.g., often populated by model provider for AIMessages)

In [1]:
# import libraries
from pprint import pprint
from langchain_core.prompts import ChatPromptTemplate

In [2]:
from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage

messages = [AIMessage(content=f"So you said you were researching ocean mammals?", name="Model")]
messages.append(HumanMessage(content=f"Yes, that's right.",name="Lance"))
messages.append(AIMessage(content=f"Great, what would you like to learn about.", name="Model"))
messages.append(HumanMessage(content=f"I want to learn about the best place to see Orcas in the US.", name="Lance"))

for m in messages:
    m.pretty_print()

================================== Ai Message ==================================
Name: Model

So you said you were researching ocean mammals?
================================ Human Message =================================
Name: Lance

Yes, that's right.
================================== Ai Message ==================================
Name: Model

Great, what would you like to learn about.
================================ Human Message =================================
Name: Lance

I want to learn about the best place to see Orcas in the US.


## Chat Models

Chat models can use a sequence of message as input and support message types, as discussed above.

There are many to choose from! Let's work with OpenAI.

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name = 'gpt-4o-mini')

In [4]:
result = llm.invoke(messages)
type(result)

langchain_core.messages.ai.AIMessage

In [5]:
result

AIMessage(content='One of the best places to see orcas in the U.S. is in the Pacific Northwest, particularly in the waters around the San Juan Islands in Washington State. The orca population there includes the famous Southern Resident killer whales, which are often spotted from late spring to early fall. \n\nAnother excellent location is the waters off the coast of Alaska, especially around areas like the Inside Passage, where orcas are frequently seen. \n\nAdditionally, you might consider Monterey Bay in California, where orcas can sometimes be spotted during their seasonal migrations. \n\nWhale watching tours in these areas can provide you with a great chance to see orcas in their natural habitat.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 132, 'prompt_tokens': 67, 'total_tokens': 199, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_t

In [6]:
result.response_metadata

{'token_usage': {'completion_tokens': 132,
  'prompt_tokens': 67,
  'total_tokens': 199,
  'completion_tokens_details': {'audio_tokens': 0,
   'reasoning_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4o-mini-2024-07-18',
 'system_fingerprint': 'fp_0ba0d124f1',
 'finish_reason': 'stop',
 'logprobs': None}

# Tools

Tools are useful whenever you want a model to interact with external systems.

External systems (e.g., APIs) often require a particular input schema or payload, rather than natural language.

When we bind an API, for example, as a tool we given the model awareness of the required input schema.

The model will choose to call a tool based upon the natural language input from the user.

And, it will return an output that adheres to the tool's schema.

Many LLM providers support tool calling and tool calling interface in LangChain is simple.

You can simply pass any Python function into ChatModel.bind_tools(function)

<img src='https://camo.githubusercontent.com/9ccbea2be12f1cb8a31c62e10a2360c30f35879c7860d66626d6aac58f648e9d/68747470733a2f2f63646e2e70726f642e776562736974652d66696c65732e636f6d2f3635623863643732383335636565616364343434396135332f3636646261623038646331633137613761353766393936305f636861696e322e706e67'>

Let's showcase a simple example of tool calling!

The multiply function is our tool

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

llm_with_tools = llm.bind_tools([multiply])

f we pass an input - e.g., "What is 2 multiplied by 3" - we see a tool call returned.

The tool call has specific arguments that match the input schema of our function along with the name of the function to call.

{'arguments': '{"a":2,"b":3}', 'name': 'multiply'}

In [9]:
tool_call = llm_with_tools.invoke([HumanMessage(content=f"What is 2 multiplied by 3", name="Lance")])
tool_call

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8zM39R49cXAL4AeD16t2fmk5', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 62, 'total_tokens': 79, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f85bea6784', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-33d727c3-bd6d-4ef4-bdc9-25ffa2e4d8b8-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_8zM39R49cXAL4AeD16t2fmk5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 62, 'output_tokens': 17, 'total_tokens': 79, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [10]:
tool_call.additional_kwargs['tool_calls']

[{'id': 'call_8zM39R49cXAL4AeD16t2fmk5',
  'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'},
  'type': 'function'}]

### Using messages as state

With these foundations in place, we can now use messages in our graph state.

Let's define our state, MessagesState, as a TypedDict with a single key: messages.

messages is simply a list of messages, as we defined above (e.g., HumanMessage, etc).